In [1]:
library(singscore)
library(tidyverse)
library(ggplot2)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
# Set random seed so this part is reproducible
# https://www.random.org/ 2023-08-09
set.seed(3866) 

In [3]:
gs0 <- read.table("../../../../data/gs/IO-Atlas-NSCLC-NSCLC-Response-Cluster-0-TPM-MinMaxNorm-TRAIN-2023-08-10-V4-gene-sets.tsv",
                 sep='\t', 
                 header=T)

gs1 <- read.table("../../../../data/gs/IO-Atlas-NSCLC-NSCLC-Response-Cluster-1-TPM-MinMaxNorm-TRAIN-2023-08-10-V4-gene-sets.tsv",
                 sep='\t', 
                 header=T)

gs2 <- read.table("../../../../data/gs/IO-Atlas-NSCLC-NSCLC-Response-Cluster-2-TPM-MinMaxNorm-TRAIN-2023-08-10-V4-gene-sets.tsv",
                 sep='\t', 
                 header=T)

gene_sets <- list(gs0, gs1, gs2)
up_gene_set_db <- list()
dwn_gene_set_db <- list()

for (i in seq(3)) {
    up_response_subtype_gs_db <- list()
    dwn_response_subtype_gs_db <- list()
    for (gene_set in unique(gene_sets[i][[1]]$gs))
        {
            
            up_response_subtype_gs_db[[gene_set]] <- gene_sets[i][[1]] %>% filter((gs == gene_set) & (direction == "up")) %>%  pull("gene") %>% unique()
            dwn_response_subtype_gs_db[[gene_set]] <- gene_sets[i][[1]] %>% filter((gs == gene_set) & (direction == "down")) %>%  pull("gene") %>% unique()
        }
    up_gene_set_db[[i]] <- up_response_subtype_gs_db
    dwn_gene_set_db[[i]] <- dwn_response_subtype_gs_db
}


In [4]:
minmax <- read.table("../../../../data/expression/processed/train/V4/IO-Atlas-NSCLC-TPM-MinMaxNorm-TRAIN-2023-08-10-V4.tsv", 
                          sep='\t', 
                          row.names=1, 
                          header=T)

labels <- read.table("../../../../data/expression/processed/train/V4/IO-Atlas-NSCLC-LABEL-TRAIN-2023-08-10-V4.tsv", 
                  sep='\t', 
                  row.names=1, 
                  header=T)

rnk <- rankGenes(minmax)

scores <- data.frame()

for (i in seq(3))
    {
        
        for (gs in names(up_gene_set_db[[i]]))
            {
                score <- simpleScore(rnk, 
                                     upSet = up_gene_set_db[[i]][[gs]],
                                     downSet = dwn_gene_set_db[[i]][[gs]])
            
                score_name <- sprintf("Response Subtype Cluster %s %s", i - 1, gs)
            
                scores[row.names(score), score_name] <- score$TotalScore
            }
    }

write.table(scores, 
             "../../../../data/enrichment/IO-Atlas-NSCLC-TPM-MinMaxNorm-TRAIN-2023-08-10-V4-response-subtype-clusters.tsv",
             sep='\t')

In [5]:
minmax <- read.table("../../../../data/expression/processed/test/V4/IO-Atlas-NSCLC-TPM-MinMaxNorm-TEST-2023-08-10-V4.tsv", 
                          sep='\t', 
                          row.names=1, 
                          header=T)

labels <- read.table("../../../../data/expression/processed/test/V4/IO-Atlas-NSCLC-LABEL-TEST-2023-08-10-V4.tsv", 
                  sep='\t', 
                  row.names=1, 
                  header=T)

rnk <- rankGenes(minmax)

scores <- data.frame()

for (i in seq(3))
    {
        
        for (gs in names(up_gene_set_db[[i]]))
            {
                score <- simpleScore(rnk, 
                                     upSet = up_gene_set_db[[i]][[gs]],
                                     downSet = dwn_gene_set_db[[i]][[gs]])
            
                score_name <- sprintf("Response Subtype Cluster %s %s", i, gs)
            
                scores[row.names(score), score_name] <- score$TotalScore
            }
    }

write.table(scores, 
             "../../../../data/enrichment/IO-Atlas-NSCLC-TPM-MinMaxNorm-TEST-2023-08-10-V4-response-subtype-clusters.tsv",
             sep='\t')